In [246]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import numpy as np
from tqdm import tqdm
import pandas as pd
from torchvision import models
from torch.optim.lr_scheduler import _LRScheduler

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr



import warnings
warnings.filterwarnings("ignore")

In [2]:
device = torch.device("mps")

In [3]:
DATA_DIR = '/Users/gufran/Desktop/PfsPredictionLungCancer/data/pathology_patches'

In [4]:
class MultiModalData(Dataset):
    def __init__(self, root_dir, df_path=None, num_patches=10, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.num_patches = num_patches
        self.slide_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.df = pd.read_csv(df_path)
        
        self.df_id = "id"
        self.df.drop(["pfs", "dmp_pt_id" if "dmp_pt_id" in self.df.columns else "pdl1_image_id"],axis=1, inplace=True)

    def __len__(self):
        return len(self.slide_dirs)

    def __getitem__(self, idx):
        slide_dir = self.slide_dirs[idx]
        pfs_label = float(slide_dir.split('_')[-1])

        patch_paths = [os.path.join(self.root_dir, slide_dir, patch) for patch in os.listdir(os.path.join(self.root_dir, slide_dir))]
        patch_paths = patch_paths[:self.num_patches]  # Take the first 'num_patches' patches

        patches = [self.transform(Image.open(patch)) for patch in patch_paths]

        while len(patches) < self.num_patches:
            patches.append(patches[-1])
        
        path_clin_features = np.array(self.df[self.df[self.df_id] == int(slide_dir.split('_')[0])].drop(["id"], axis=1).iloc[0])
        return {'patches': torch.stack(patches), 'pfs_label': torch.tensor(pfs_label, dtype=torch.float32), "pcf": torch.tensor(path_clin_features, dtype=torch.float32)}

In [5]:
transform = transforms.Compose([
    # transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [23]:
batch_size = 8

train_dataset = MultiModalData(root_dir=os.path.join(DATA_DIR, "train"), df_path="/Users/gufran/Desktop/PfsPredictionLungCancer/data/train_path_clin_filtered.csv", transform=transform)
val_dataset = MultiModalData(root_dir=os.path.join(DATA_DIR, "val"), df_path="/Users/gufran/Desktop/PfsPredictionLungCancer/data/val_path_clin_filtered.csv", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [7]:
next(iter(train_loader))["patches"].shape

torch.Size([8, 10, 3, 128, 128])

In [133]:
class DataFusionModel(nn.Module):
    def __init__(self, currently_training=True):
        super(DataFusionModel, self).__init__()
        self.currently_training = currently_training
        
        self.resnet = models.resnet50(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        
        self.mnet = models.mobilenet_v2(pretrained=True)
        mobilenet_in_features = self.mnet.classifier[1].in_features
        self.mnet.classifier = nn.Identity()
        
        self.fc = nn.Linear(in_features + mobilenet_in_features + 15, 256)
        
        self.regressor = nn.Sequential(
            nn.ReLU(),
            nn.Linear(256, 1)
        )
        
    def forward(self, img_batch, pcf_batch):
        image_features_resnet = []
        for patches in img_batch:
            patch_outputs = [self.resnet(patch.unsqueeze(0)) for patch in patches]
            aggregated_output = torch.stack(patch_outputs).mean(dim=0)
            aggregated_output = aggregated_output.squeeze()
            image_features_resnet.append(aggregated_output)
            
        image_features_mnet = []
        for patches in img_batch:
            patch_outputs = [self.mnet(patch.unsqueeze(0)) for patch in patches]
            aggregated_output = torch.stack(patch_outputs).mean(dim=0)
            aggregated_output = aggregated_output.squeeze()
            image_features_mnet.append(aggregated_output)
        
        multimodal_features = []
        for i in range(len(img_batch)):
            multimodal_features.append(torch.cat((image_features_resnet[i], image_features_mnet[i], pcf_batch[i]), dim=0))
        
        multimodal_features = torch.stack(multimodal_features)

        final_features = self.fc(multimodal_features)
        if self.currently_training: final_features = self.regressor(final_features)
        # else: final_features = F.relu(final_features)
        
        return final_features

In [26]:
model = DataFusionModel().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

num_epochs = 100

In [27]:
class LinearDecayLR(_LRScheduler):
    def __init__(self, optimizer, n_epoch, start_decay, last_epoch=-1):
        self.start_decay=start_decay
        self.n_epoch=n_epoch
        super(LinearDecayLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        last_epoch = self.last_epoch
        n_epoch=self.n_epoch
        b_lr=self.base_lrs[0]
        start_decay=self.start_decay
        if last_epoch>start_decay:
            lr=b_lr-b_lr/(n_epoch-start_decay)*(last_epoch-start_decay)
        else:
            lr=b_lr
        return [lr]
    
lr_scheduler=LinearDecayLR(optimizer, num_epochs, 10)

In [11]:
for epoch in range(num_epochs):
    model.train()

    y_true_train = []
    y_pred_train = []
    
    best_mse = 9999999
    for batch in tqdm(train_loader):
        patches = batch['patches'].to(device)
        pfs_labels = batch['pfs_label'].to(device)
        pcf_data = batch['pcf'].to(device)

        optimizer.zero_grad()
        outputs = model(patches, pcf_data)
        loss = criterion(outputs, pfs_labels)
        loss.backward()
        optimizer.step()

        y_true_train.extend(pfs_labels.cpu().numpy())
        
        for pred_val in outputs.detach().cpu().numpy():
            y_pred_train.append(float(pred_val))
      
    lr_scheduler.step()
    
    mse_train = mean_squared_error(y_true_train, y_pred_train)
    pearson_corr_train, _ = pearsonr(y_true_train, y_pred_train)
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"MSE: {mse_train} - Pearson Correlation: {pearson_corr_train}")
    print(f"Learning Rate: {lr_scheduler.get_lr()}", end="\n\n")
    
    if mse_train<best_mse:
        best_mse = mse_train
        torch.save(model.state_dict(), f"/Users/gufran/Desktop/PfsPredictionLungCancer/notebooks/checkpoint/multimodal.pt")

100%|███████████████████████████████████████████| 12/12 [01:04<00:00,  5.38s/it]


Epoch [1/100]
MSE: 174.23263907077663 - Pearson Correlation: 0.1250376785978919
Learning Rate: [5e-05]



100%|███████████████████████████████████████████| 12/12 [01:02<00:00,  5.17s/it]


Epoch [2/100]
MSE: 103.6848539237711 - Pearson Correlation: 0.0798068383199664
Learning Rate: [5e-05]



100%|███████████████████████████████████████████| 12/12 [01:04<00:00,  5.37s/it]


...

Epoch [98/100]
MSE: 71.53693732162591 - Pearson Correlation: 0.833741445759859
Learning Rate: [1.1111111111111057e-06]



100%|███████████████████████████████████████████| 12/12 [01:10<00:00,  5.87s/it]


Epoch [99/100]
MSE: 71.51733206104463 - Pearson Correlation: 0.8353437301656723
Learning Rate: [5.555555555555495e-07]



100%|███████████████████████████████████████████| 12/12 [01:04<00:00,  5.38s/it]


Epoch [100/100]
MSE: 71.49900769051669 - Pearson Correlation: 0.8349957451865692
Learning Rate: [-6.776263578034403e-21]



In [30]:
model = DataFusionModel().to(device)
model.load_state_dict(torch.load("/Users/gufran/Desktop/PfsPredictionLungCancer/notebooks/checkpoint/multimodal.pt"))

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

num_epochs = 100

lr_scheduler=LinearDecayLR(optimizer, num_epochs, 10)

for epoch in range(num_epochs):
    model.train()

    y_true_train = []
    y_pred_train = []
    
    best_mse = 71.49900769051669
    for batch in tqdm(train_loader):
        patches = batch['patches'].to(device)
        pfs_labels = batch['pfs_label'].to(device)
        pcf_data = batch['pcf'].to(device)

        optimizer.zero_grad()
        outputs = model(patches, pcf_data)
        loss = criterion(outputs, pfs_labels)
        loss.backward()
        optimizer.step()

        y_true_train.extend(pfs_labels.cpu().numpy())
        
        for pred_val in outputs.detach().cpu().numpy():
            y_pred_train.append(float(pred_val))
      
    lr_scheduler.step()
    
    mse_train = mean_squared_error(y_true_train, y_pred_train)
    pearson_corr_train, _ = pearsonr(y_true_train, y_pred_train)
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"MSE: {mse_train} - Pearson Correlation: {pearson_corr_train}")
    print(f"Learning Rate: {lr_scheduler.get_lr()}", end="\n\n")
    
    if mse_train<best_mse:
        best_mse = mse_train
        torch.save(model.state_dict(), f"/Users/gufran/Desktop/PfsPredictionLungCancer/notebooks/checkpoint/multimodal_post.pt")

100%|███████████████████████████████████████████| 12/12 [01:05<00:00,  5.43s/it]


Epoch [1/100]
MSE: 72.04992576772663 - Pearson Correlation: 0.6396412682269137
Learning Rate: [1e-05]



100%|███████████████████████████████████████████| 12/12 [01:04<00:00,  5.40s/it]


Epoch [2/100]
MSE: 71.40840023543183 - Pearson Correlation: 0.8058354704341856
Learning Rate: [1e-05]



100%|███████████████████████████████████████████| 12/12 [01:07<00:00,  5.60s/it]


...

Epoch [98/100]
MSE: 69.19580020934723 - Pearson Correlation: 0.9154263682197712
Learning Rate: [2.2222222222222217e-07]



100%|███████████████████████████████████████████| 12/12 [01:04<00:00,  5.41s/it]


Epoch [99/100]
MSE: 69.18132682285652 - Pearson Correlation: 0.9156773205400714
Learning Rate: [1.1111111111111024e-07]



100%|███████████████████████████████████████████| 12/12 [01:05<00:00,  5.49s/it]


Epoch [100/100]
MSE: 69.18152593553712 - Pearson Correlation: 0.9157353772478032
Learning Rate: [0.0]



In [429]:
model = DataFusionModel().to(device)
# model.load_state_dict(torch.load("/Users/gufran/Desktop/PfsPredictionLungCancer/notebooks/checkpoint/multimodal.pt"))
model.load_state_dict(torch.load("/Users/gufran/Desktop/PfsPredictionLungCancer/notebooks/checkpoint/multimodal_post.pt"))

<All keys matched successfully>

In [430]:
model.eval()
y_true_val = []
y_pred_val = []

for batch in tqdm(val_loader):
    patches = batch['patches'].to(device)
    pfs_labels = batch['pfs_label'].to(device)
    pcf_data = batch['pcf'].to(device)

    outputs = model(patches, pcf_data)
    y_true_val.extend(pfs_labels.cpu().numpy())
    
    for pred_val in outputs.detach().cpu().numpy():
        y_pred_val.append(float(pred_val))

mse_val = mean_squared_error(y_true_val, y_pred_val)
pearson_corr_val, _ = pearsonr(y_true_val, y_pred_val)
print(f"MSE: {mse_val} - Pearson Correlation: {pearson_corr_val}")

 43%|███████████████████▎                         | 3/7 [00:05<00:07,  1.95s/it]


KeyboardInterrupt: 

In [431]:
def create_df(model, loader):
    model_ = DataFusionModel(currently_training=False).to(device)
    model_.load_state_dict(model.state_dict())

    model_.eval()

    features_list = []
    pfs_values_list = []
    for batch in tqdm(loader):
        patches = batch['patches'].to(device)
        pfs_labels = batch['pfs_label'].to(device)
        pcf_data = batch['pcf'].to(device)

        with torch.no_grad():
            outputs = model_(patches, pcf_data)
            
            for o in outputs:
                features_list.append(list(o.cpu().numpy()))
        
        pfs_values_list.extend(pfs_labels.cpu().numpy())
        
    df = pd.DataFrame(data=features_list, columns=[f'feature_{i}' for i in range(len(features_list[0]))])
    df['pfs'] = pfs_values_list

    return df

model = DataFusionModel()
# model.load_state_dict(torch.load("/Users/gufran/Desktop/PfsPredictionLungCancer/notebooks/checkpoint/multimodal.pt"))
model.load_state_dict(torch.load("/Users/gufran/Desktop/PfsPredictionLungCancer/notebooks/checkpoint/multimodal_post.pt"))

dftr = create_df(model, train_loader)
dfte = create_df(model, val_loader)

100%|█████████████████████████████████████████████| 7/7 [00:10<00:00,  1.44s/it]


In [432]:
dftr.shape,dfte.shape

((94, 257), (50, 257))

In [312]:
from sklearn.preprocessing import MinMaxScaler

In [369]:
# dftr.to_csv("/Users/gufran/Desktop/PfsPredictionLungCancer/data/multimodal_train.csv", index=False)
# dfte.to_csv("/Users/gufran/Desktop/PfsPredictionLungCancer/data/multimodal_val.csv", index=False)

In [433]:
correlation_threshold = 0.01
correlations = dftr.drop("pfs", axis=1).corrwith(dftr['pfs'])
selected_columns = correlations[(correlations >= correlation_threshold) | (correlations <= -correlation_threshold)].index

selected_columns = list(selected_columns)
# selected_columns

In [434]:
for sc in selected_columns:
    if sc in ["dmp_pt_id", "pfs"]: continue
    print(f"Correlation with {sc}: {dftr['pfs'].corr(dftr[sc])}")

Correlation with feature_0: 0.10029362259466426
Correlation with feature_1: -0.13271471427599973
Correlation with feature_2: -0.09982973758072936
Correlation with feature_3: -0.12855659023538688
Correlation with feature_4: 0.11845733146837749
Correlation with feature_5: -0.12640003813339756
Correlation with feature_6: 0.1197360294789298
Correlation with feature_7: -0.12649468462515984
Correlation with feature_8: -0.13113971337685268
Correlation with feature_9: 0.15578891084159424
Correlation with feature_10: 0.1393835395433827
Correlation with feature_11: -0.15029075916342893
Correlation with feature_12: -0.17499163926315653
Correlation with feature_13: -0.1303680371988665
Correlation with feature_14: -0.12325858143177537
Correlation with feature_15: -0.12997120196023348
Correlation with feature_16: -0.14257498006767808
Correlation with feature_17: -0.14664703979795815
Correlation with feature_18: 0.15611935369007796
Correlation with feature_19: 0.13629201945111172
Correlation with fea

In [435]:
# dftr_s = dftr[selected_columns+["pfs"]]
# dfte_s = dfte[selected_columns+["pfs"]]

dftr_s = dftr
dfte_s = dfte

In [436]:
X_train, y_train = dftr_s.drop(["pfs"], axis=1), dftr_s["pfs"]
X_test, y_test = dfte_s.drop(["pfs"], axis=1), dfte_s["pfs"]

In [437]:
classifiers_reg = {
    'SVM': {
        'name': 'Support Vector Machine',
        'classifier': SVR(),
        'param_grid': {'C': [0.1, 1.0, 5.0, 10.0], 'kernel': ['linear', 'rbf']}
    },
    'XGBoost': {
        'name': 'XGBoost',
        'classifier': XGBRegressor(),
        'param_grid': {'n_estimators': [50, 100, 200], 'max_depth': [3, 4, 5]}
    },
    'AdaBoost': {
        'name': 'AdaBoost',
        'classifier': AdaBoostRegressor(),
        'param_grid': {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.5, 1.0]}
    },
    'RandomForest': {
        'name': 'Random Forest',
        'classifier': RandomForestRegressor(),
        'param_grid': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
    },
    'DecisionTree': {
        'name': 'Decision Tree',
        'classifier': DecisionTreeRegressor(),
        'param_grid': {'max_depth': [None, 10, 20]}
    }
}

In [401]:
# best_models = {}

# for clf_name, clf_info in classifiers_reg.items():
#     print(f"Performing GridSearchCV for {clf_info['name']}...")
    
#     clf = clf_info['classifier']
#     param_grid = clf_info['param_grid']
    
#     grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
#     grid_search.fit(X_train, y_train)

#     best_models[clf_name] = grid_search.best_estimator_

# print()

# for clf_name, best_model in best_models.items():
#     print(f"Evaluating {clf_name} on test data...")
    
#     y_pred = best_model.predict(X_test)

#     mse = mean_squared_error(y_test, y_pred)
#     pearson_coefficient, _ = pearsonr(y_test, y_pred)
    
#     print(f"Mean Squared Error on Test Data: {mse:.4f}")
#     print(f"Pearson Correlation Coefficient on Test Data: {pearson_coefficient:.4f}\n")

In [441]:
from sklearn.ensemble import BaggingRegressor

bagging_reg = BaggingRegressor(base_estimator=SVR(kernel='rbf', epsilon=0.2, C=1.0), n_estimators=2, random_state=42)  # You can customize parameters
bagging_reg.fit(X_train, y_train)

y_pred_bagging = bagging_reg.predict(X_test)
mse_bagging = mean_squared_error(y_test, y_pred_bagging)
pearson_coefficient_bagging, _ = pearsonr(y_test, y_pred_bagging)

# if mse_bagging<40:
print(f"Mean Squared Error on Test Data (Bagging): {mse_bagging:.4f}")
print(f"Pearson Correlation Coefficient on Test Data (Bagging): {pearson_coefficient_bagging:.4f}\n")

Mean Squared Error on Test Data (Bagging): 35.6142
Pearson Correlation Coefficient on Test Data (Bagging): 0.3929



In [287]:
best_models["BaggingRegressor"] = bagging_reg

In [288]:
best_models.items()

dict_items([('SVM', SVR(C=0.1)), ('AdaBoost', AdaBoostRegressor(learning_rate=0.1, n_estimators=100)), ('RandomForest', RandomForestRegressor(max_depth=10, n_estimators=200)), ('DecisionTree', DecisionTreeRegressor()), ('BaggingRegressor', BaggingRegressor(base_estimator=SVR(), n_estimators=2, random_state=42))])

In [388]:
from sklearn.ensemble import VotingRegressor

voting_reg = VotingRegressor(estimators=list(best_models.items()))
voting_reg.fit(X_train, y_train)

y_pred_voting = voting_reg.predict(X_test)
mse_voting = mean_squared_error(y_test, y_pred_voting)
pearson_coefficient_voting, _ = pearsonr(y_test, y_pred_voting)

print(f"Mean Squared Error on Test Data (Voting): {mse_voting:.4f}")
print(f"Pearson Correlation Coefficient on Test Data (Voting): {pearson_coefficient_voting:.4f}\n")

Mean Squared Error on Test Data (Voting): 67.7944
Pearson Correlation Coefficient on Test Data (Voting): -0.0827

